In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

# ACS API Queries
Variables: https://www.census.gov/data/developers/data-sets/acs-5year.2016.html#list-tab-1806015614

In [2]:
import requests
import json
df_model = pd.DataFrame()
yr_df_list = []
host = 'https://api.census.gov/data'
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']
dataset_acronym = '/acs/acs5'
g = '?get='
variables_v2 = 'NAME,B02001_001E,B19051_001E,B19057_002E,B19058_002E,C16002_002E,B07001_049E,B07001_033E,B08006_008E,B08006_002E,B08006_014E,B08006_015E,B08006_017E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B03001_003E,B24031_001E,B24031_002E,B24031_005E,B24031_008E,B24031_013E,B24031_016E,B24031_020E,B24031_023E,B06009_005E,B06009_006E,B05001_002E,B05001_005E,B05001_006E,B07001_017E,B07001_065E,B19013_001E,B25064_001E,B25035_001E,B25039_001E,B01002_001E,B25036_013E,B25077_001E,B25040_004E,B25040_008E'
variables_v1 = 'NAME,B02001_001E,B19051_001E,B19057_002E,B19058_002E,B16002_002E,B07001_049E,B07001_033E,B08006_008E,B08006_002E,B08006_014E,B08006_015E,B08006_017E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B03001_003E,B24031_001E,B24031_002E,B24031_005E,B24031_008E,B24031_013E,B24031_016E,B24031_020E,B24031_023E,B06009_005E,B06009_006E,B05001_002E,B05001_005E,B05001_006E,B07001_017E,B07001_065E,B19013_001E,B25064_001E,B25035_001E,B25039_001E,B01002_001E,B25036_013E,B25077_001E,B25040_004E,B25040_008E'

location = '&for=TRACT:*&in=state:36%county:005,047,061,081,085'
usr_key = f"&key=332de56715b7f43a68ee347091c29afad759be55"

for year in years:
    print(year)
    if year in ['2010','2011','2012','2013','2014','2015','2022']:
        variables = variables_v1
    else:
        variables = variables_v2
    query_url = f"{host}/{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
    
    # Use requests package to call out to the API
    response = requests.get(query_url)

    ## Rename columns for clarity
    table_lists = json.loads(response.text)
    df = pd.DataFrame(table_lists[1:], columns=table_lists[0])
    df['year'] = year

    col_names = {"B02001_001E":"total_pop",
                 "B02001_002E":"percent_white",
                 "B02001_003E":"percent_black",
                 "B02001_004E":"percent_native",
                 "B02001_005E":"percent_asian",
                 "B02001_006E":"percent_pacific",
                 "B03001_003E":"percent_latino",
                 "B24031_001E":"total_num_industry",  # total CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER
                 "B24031_002E":"percent_work_agriculture",  # Total:!!Agriculture, forestry, fishing and hunting, and mining:
                 "B24031_005E":"percent_work_construction",
                 "B24031_008E":"percent_work_retail",
                 "B24031_013E":"percent_work_finance",
                 "B24031_016E":"percent_work_stem",  # Professional, scientific, and management, and administrative, and waste management services:
                 "B24031_020E":"percent_work_edu_health",  # al:!!Educational services, and health care and social assistance:
                 "B24031_023E":"percent_work_art",  # Arts, entertainment, and recreation, and accommodation and food services:
                 "B06009_005E":"percent_bachelors",  # num ppl with bachelors degree
                 "B06009_006E":"percent_grad",
                 "B18140_002E":"percent_disabled",  # num ppl With a disability:
                 "B05001_002E":"percent_born_citizen",
                 "B05001_005E":"percent_naturalized_citizen",
                 "B05001_006E":"percent_not_citizen",
                 "B07001_017E":"percent_same_house_1yr",  # Estimate!!Total:!!Same house 1 year ago:
                 "B07001_065E":"percent_out_of_state",  # Estimate!!Total:!!Moved from different state: (in past year)
                 "B07001_049E":"percent_out_of_county",  # Estimate!!Total:!!Moved from different county within same state (past year)
                 "B07001_033E":"percent_moved_within_county",  # !Total:!!Moved within same county (past year)
                 "B08006_008E":"percent_public_transit",  # !!Total:!!Public transportation to work (excluding taxicab)
                 "B08006_002E":"percent_drive_commute",  # !!Total:!!drive to work
                 "B08006_014E":"percent_bike_commute",
                 "B08006_015E":"percent_walk_commute",  # num ppl walk to work
                 "B08006_017E":"percent_wfh",  # num ppl work from home
                 "B19013_001E":"med_income",  # Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
                 "B25064_001E":"med_rent_acs",  # Estimate!!Median gross rent
                 "B25035_001E":"med_yr_built",  # !Median year structure built
                 "B01002_001E":"med_age",
                 "B25036_013E":"percent_hh_rented",  # num hh renter occupied
                 "B25077_001E":"med_value",  # median home value
                 "B25039_001E":"med_yr_moved_in",  # Estimate!!Median year householder moved into unit rent or own
                 "B19051_001E":"total_num_hh",
                 "C16002_002E":"percent_hh_english",  # num hh speaking english only  #### not in 2016
                 "B16002_002E":"percent_hh_english",  # num hh speaking english only
                 "B19058_002E":"percent_hh_snap",  # Estimate!!Total:!!Household received Food Stamps/SNAP in the past 12 months:
                 "B19057_002E":"percent_hh_income_assist",  # PUBLIC ASSISTANCE INCOME IN THE PAST 12 MONTHS FOR HOUSEHOLDS
                 "B25040_004E":"percent_hh_electric",  # Estimate!!Total:!!Electricity
                 "B25040_008E":"percent_hh_solar"}  # Estimate!!Total:!!Solar energy
        
    df.rename(columns= col_names, inplace=True)

    # change counts into percentages
    for col in ['percent_out_of_county',
       'percent_moved_within_county', 'percent_public_transit',
       'percent_drive_commute', 'percent_bike_commute', 'percent_walk_commute',
       'percent_wfh','percent_white', 'percent_black',
       'percent_native', 'percent_asian', 'percent_pacific', 'percent_latino','percent_bachelors', 'percent_grad',
       'percent_born_citizen', 'percent_naturalized_citizen',
       'percent_not_citizen', 'percent_same_house_1yr', 'percent_out_of_state']:
    
        df[col] = df[col].astype(float)/df['total_pop'].astype(float)
    
    for col in ['percent_work_agriculture',
       'percent_work_construction', 'percent_work_retail',
       'percent_work_finance', 'percent_work_stem', 'percent_work_edu_health',
       'percent_work_art']:
        df[col] = df[col].astype(float)/df['total_num_industry'].astype(float)

    for col in ['percent_hh_income_assist',
       'percent_hh_snap', 'percent_hh_english','percent_hh_rented','percent_hh_electric', 'percent_hh_solar']:
        df[col] = df[col].astype(float)/df['total_num_hh'].astype(float)

    # change year into number of years from present
    df['med_yr_built'] = df['med_yr_built'].replace("-",np.nan)
    df['med_yr_built'] = df['med_yr_built'].apply(lambda x: str(x).replace("-",""))
    df['med_yr_built'] = df['med_yr_built'].apply(lambda x: str(x).replace("+",""))
    current_year = datetime.now().year
    df['med_hh_age'] = current_year - df['med_yr_built'].astype(float)
    
    df['med_yr_moved_in']= df['med_yr_moved_in'].replace("-",np.nan)
    df['med_yr_moved_in'] = df['med_yr_moved_in'].apply(lambda x: str(x).replace("+",""))
    current_year = datetime.now().year
    df['med_hh_tenure'] = current_year - df['med_yr_moved_in'].astype(float)

    df['full_tract'] = df.county + df.tract
    
    df.drop(["med_yr_moved_in","med_yr_built","state","total_num_hh","total_num_industry","county","tract"], axis=1, inplace=True)

    # JOIN THIS YEAR INTO THE REST  TODO: need to merge instead so every year tract is accounted for in every year
    yr_df_list.append(df)  


2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [73]:
# loop through each df and join to get full tracts series
all_tracts = yr_df_list[0]
for yr_df in yr_df_list[1:]:
    all_tracts = all_tracts[['full_tract']].merge(yr_df[['full_tract']], how='outer').drop_duplicates()

# loop through again and join full tracts series with each df
to_concat=[]
for yr_df in yr_df_list:
    yr_df = yr_df.merge(all_tracts, how='outer')
    # fill in the null years
    yr_df['year'] = yr_df['year'].ffill()
    to_concat.append(yr_df)

# concat all df's
df_model = pd.concat(to_concat)


In [74]:
# # sort dataframe on tracts and years, then backfill missing vals
# sorted = df_model.sort_values(['full_tract','year']).reset_index(drop=True)
# df_model = sorted.groupby('full_tract').bfill().reset_index(drop=True)
# df_model = pd.concat([sorted['full_tract'], df_model], axis=1)


In [75]:


### TODO -> address -66666666 values later!!!
df_model['med_value'] = df_model['med_value'].astype(float)
df_model['med_age'] = df_model['med_age'].astype(float)
df_model['med_rent_acs'] = df_model['med_rent_acs'].astype(float)
df_model['med_income'] = df_model['med_income'].astype(float)
df_model['total_pop'] = df_model['total_pop'].astype(float)

# add GEOID column for mapping
df_model['GEOID'] = '36'+df_model['full_tract']

In [76]:
df_model.sample(5)

,NAME,total_pop,percent_hh_income_assist,percent_hh_snap,percent_hh_english,percent_out_of_county,percent_moved_within_county,percent_public_transit,percent_drive_commute,percent_bike_commute,...,med_age,percent_hh_rented,med_value,percent_hh_electric,percent_hh_solar,year,med_hh_age,med_hh_tenure,full_tract,GEOID
1255,"Census Tract 32, New York County, New York",8000.0,0.014245,0.060267,0.791804,0.049125,0.034625,0.321750,0.041125,0.045500,...,40.5,0.763533,-666666666.0,0.314267,0.0,2021,2025.0,16.0,061003200,36061003200
478,"Census Tract 110, Kings County, New York",2482.0,0.039706,0.229412,0.160294,0.000000,0.047945,0.325947,0.074537,0.025383,...,28.2,0.000000,608100.0,0.033824,0.0,2014,86.0,19.0,047011000,36047011000
181,"Census Tract 237.03, Bronx County, New York",5527.0,0.137390,0.441656,0.212673,0.011760,0.043061,0.272661,0.029673,0.000000,...,28.5,0.993726,-666666666.0,0.086575,0.0,2018,2025.0,17.0,005023703,36005023703
1478,"Census Tract 215, New York County, New York",4128.0,0.032765,0.184300,0.653925,0.014535,0.043362,0.234981,0.044331,0.009932,...,28.8,0.984983,583300.0,0.271672,0.0,2016,52.0,18.0,061021500,36061021500
125,"Census Tract 183.02, Bronx County, New York",4176.0,0.049451,0.375981,0.408163,0.000000,0.101293,0.266044,0.046935,0.014368,...,31.0,0.000000,531300.0,0.063579,0.0,2013,86.0,24.0,005018302,36005018302


In [77]:
# df_model.to_parquet("Data/Cleaned/ACS.parquet")